In [1]:
from utils import load_csv
from columns import *

df = load_csv("../data/intermediates/sepsis_cohort/states_and_actions_filled.csv")

print("Before filtering:", len(set(df[C_ICUSTAYID])), "ICU stays")  # count before

Before filtering: 32608 ICU stays


In [2]:
# #### exclude early deaths from possible withdrawals ####
# stats per patient
q = df[C_BLOC] == 1
# fence_posts=find(q(:,1)==1);
num_of_trials = len(set(df[C_ICUSTAYID]))  # size(fence_posts,1);
a = df[[C_BLOC, C_ICUSTAYID, C_MORTA_90, C_MAX_DOSE_VASO, C_SOFA]]
grouped = a.groupby(C_ICUSTAYID)
d = pd.merge(grouped.agg('max'),
             grouped.size().rename(C_NUM_BLOCS),
             how='left',
             left_index=True,
             right_index=True).drop(C_BLOC, axis=1)
last_bloc = a.sort_values(C_BLOC, ascending=False).drop_duplicates(C_ICUSTAYID).rename({
    C_MAX_DOSE_VASO: C_LAST_VASO,
    C_SOFA: C_LAST_SOFA
}, axis=1).drop(C_MORTA_90, axis=1)
d = pd.merge(d,
             last_bloc,
             how='left',
             left_index=True,
             right_on=C_ICUSTAYID).set_index(C_ICUSTAYID, drop=True)
# d['GroupCount'] = a.groupby(C_ICUSTAYID).size().reset_index(drop=True)
# print(a.groupby(C_ICUSTAYID).size().index)
# print(d.index)
# d['last_max_dose_vaso'] = a.groupby(C_ICUSTAYID).agg({C_MAX_DOSE_VASO: 'last'})[C_MAX_DOSE_VASO].reset_index(drop=True)
d

,morta_90,max_dose_vaso,SOFA,num_blocs,bloc,last_vaso,last_SOFA
icustayid,,,,,,,
30000153,0,NaN,2,1,1,NaN,2
30000484,1,0.050,4,10,10,0.016,3
30000646,1,NaN,2,16,16,NaN,1
30001148,0,0.225,3,7,7,NaN,1
30001446,0,0.080,9,8,8,NaN,8
...,...,...,...,...,...,...,...
39996783,1,2.252,3,1,1,2.252,3
39996867,0,NaN,5,9,9,NaN,2
39998622,0,NaN,6,8,8,NaN,2


In [4]:
stopped_treatment = d[
    (d[C_MORTA_90] == 1) & 
    (pd.isna(d[C_LAST_VASO]) | (d[C_LAST_VASO] < 0.01)) &
    (d[C_MAX_DOSE_VASO] > 0.3) &
    (d[C_LAST_SOFA] >= d[C_SOFA] / 2) &
    (d[C_NUM_BLOCS] < 20)
].index
print(len(stopped_treatment), "stays to remove because treatment was stopped and patient died")

df = df[~df[C_ICUSTAYID].isin(stopped_treatment)]

189 stays to remove because treatment was stopped and patient died


In [14]:
# finds patients who match our criteria
# todo make more efficient (VS)
from tqdm import tqdm

e = pd.DataFrame([False] * num_of_trials)
for i in tqdm(range(num_of_trials)):
    row = d.iloc[i]
    if row[C_MORTA_90] == 1:
        ii = (df.icustayid == row.name) & (df.bloc == row['GroupCount'])  # last row for this patient
        if sum(ii) > 0:
            e.loc[i, 0] = ((df.max_dose_vaso[ii].values.tolist()[0] == 0) & (row[C_MAX_DOSE_VASO] > 0.3) & (
                        df.SOFA[ii].values.tolist()[0] >= row[C_SOFA] / 2))
        else:
            e.loc[i, 0] = False
    if i == 100: break
r = d.id[e[0] & (d.GroupCount < 20)]  # ids to be removed
# ii=ismember(reformat4t.icustayid,r)


  0%|          | 83/32608 [00:10<1:08:58,  7.86it/s]


KeyboardInterrupt: 

In [5]:
# exclude patients who died in ICU during data collection period
died_in_icu = df[
    (df[C_DIED_WITHIN_48H_OF_OUT_TIME] == 1) &
    (df[C_DELAY_END_OF_RECORD_AND_DISCHARGE_OR_DEATH] < 24)
][C_ICUSTAYID].unique()
print(len(died_in_icu), "patients to remove because died in ICU during data collection")
df = df[~df[C_ICUSTAYID].isin(died_in_icu)]

print(len(set(df[C_ICUSTAYID])), "ICU stays after filtering")  # count after



# print("Write")
# df.to_csv(args.output, index=False, float_format='%g')


1111 patients to remove because died in ICU during data collection
31308 ICU stays after filtering


In [6]:
states = pd.read_csv("../data/intermediates/mdp/patient_states.csv")
states["mechvent"]

0           NaN
1           NaN
2           NaN
3             1
4          True
           ... 
2727819       1
2727820     NaN
2727821     NaN
2727822     NaN
2727823       1
Name: mechvent, Length: 2727824, dtype: object